# NumPy Basics — 45 Worked Examples (15 per section)
This notebook contains three sections:
1) **ndarray creation, shape, dtype** (15 examples)
2) **Array indexing, slicing, reshaping** (15 examples)
3) **Broadcasting and vectorized operations** (15 examples)

**Format for each example:**
- *Markdown*: what the function is about
- *Code cell*: runnable example
- *Markdown*: options to optimize/improve usage

> Tip: Run cells top-to-bottom. You can `pip install numpy` if needed in your own environment.


In [ ]:
import numpy as np
np.__version__

## Section 1 — ndarray creation, shape, dtype (15 Examples)

### np.array
Create an array from a Python list/tuple. Controls dtype via `dtype=`. Copies data by default.

In [ ]:
a = np.array([1, 2, 3], dtype=np.int32)
print(a, a.dtype, a.shape)

**Optimization options**
- - Prefer `np.asarray` over `np.array` when you want to avoid unnecessary copies.
- Specify `dtype` to avoid implicit upcasts.

### np.asarray
Like `np.array` but avoids copying if input is already an ndarray and dtype matches.

In [ ]:
b = np.asarray([1, 2, 3])
print(b, type(b))

**Optimization options**
- - Zero-copy if feeding an ndarray. Use `dtype=` to enforce a type only when needed.

### np.zeros
Create an array filled with zeros. Useful for preallocating buffers.

In [ ]:
z = np.zeros((2,3), dtype=np.float64)
print(z)

**Optimization options**
- - Use appropriate `dtype` (e.g., `float32`) to save memory; consider `order='C'` or `'F'` for BLAS-optimized ops.

### np.ones
Create an array filled with ones. Often used for adding biases or constants.

In [ ]:
o = np.ones((2,3), dtype=np.float32)
print(o)

**Optimization options**
- - Uninitialized arrays are fastest; always write to all entries before reading to avoid undefined values.

### np.empty
Allocate array without initializing entries (may contain arbitrary memory). Fast but uninitialized.

In [ ]:
e = np.empty((3,3))
print(e)

**Optimization options**
- - If filled with one constant, `np.full` may be clearer and sometimes faster than `ones*const`.

### np.full
Create an array filled with a constant value.

In [ ]:
f = np.full((2,4), fill_value=7, dtype=np.int16)
print(f)

**Optimization options**
- - Use integer dtype that fits your range; prefer vectorized generation to Python loops.

### np.arange
Evenly spaced values within half-open interval `[start, stop)` with step.

In [ ]:
r = np.arange(0, 10, 2, dtype=np.int64)
print(r)

**Optimization options**
- - `linspace` is ideal for stable spacing; set `dtype` to control precision.

### np.linspace
Evenly spaced values over a closed interval `[start, stop]` with num points.

In [ ]:
ls = np.linspace(0.0, 1.0, num=5)
print(ls)

**Optimization options**
- - Prefer for multiplicative scales; be mindful of large exponents and floating overflow.

### np.logspace
Evenly spaced on a log scale between `10**start` and `10**stop`.

In [ ]:
lgs = np.logspace(0, 3, num=4)
print(lgs)

**Optimization options**
- - Use `eye` for linear algebra setups; consider `np.identity(n)` as a shorthand for square matrices.

### np.eye
Create a 2-D identity matrix with ones on the diagonal.

In [ ]:
I = np.eye(4, dtype=float)
print(I)

**Optimization options**
- - `diag` can both build and extract; combine with `einsum`/`matmul` for diag operations efficiently.

### np.diag
Extract a diagonal or construct a diagonal matrix from a 1-D array.

In [ ]:
d = np.diag([1,2,3])
print(d)
print(np.diag(d))

**Optimization options**
- - Avoid repeated casts in tight loops; choose the right dtype upfront.

### astype
Cast/convert array to a different dtype. Returns a new array (copy).

In [ ]:
x = np.array([1.2, 2.7, 3.1])
print(x.astype(np.int64))

**Optimization options**
- - Using correct shape early avoids later reshapes; `itemsize` helps with memory planning.

### Attributes: shape/ndim/size/itemsize
Array metadata that describe memory layout and dimensionality.

In [ ]:
a = np.arange(24).reshape(2,3,4)
print('shape:', a.shape, 'ndim:', a.ndim, 'size:', a.size, 'itemsize:', a.itemsize)

**Optimization options**
- - Function should be vectorized; avoid Python-side loops; use broadcasting inside the function.

### np.fromfunction
Construct array by executing a function over each coordinate.

In [ ]:
def f(i, j):
    return i + j
M = np.fromfunction(f, (3,4), dtype=int)
print(M)

**Optimization options**
- - `frombuffer` avoids copies; ensure alignment and correct `dtype`; beware of mutability and lifetime.

## Section 2 — Array indexing, slicing, reshaping (15 Examples)

### Basic indexing
Access elements using zero-based indices for each axis.

In [ ]:
a = np.arange(12).reshape(3,4)
print(a)
print(a[1,2])  # row 1, col 2

**Optimization options**
- - Combine scalar and tuple indexing for clarity; avoid chained indexing in favor of tuples.

### Slicing
Use `start:stop:step` per axis; slices are views (no copy) whenever possible.

In [ ]:
b = np.arange(10)
print(b[2:8:2])
M = np.arange(16).reshape(4,4)
print(M[1:3, 0:3])

**Optimization options**
- - Prefer slicing views to avoid copies; keep strides contiguous for downstream BLAS ops.

### Negative indices
Count from the end with negative indices.

In [ ]:
c = np.arange(5)
print(c[-1], c[-2])

**Optimization options**
- - Mix negative with positive indices carefully; avoid out-of-bounds checks in Python loops by vectorizing.

### Boolean indexing
Select elements using a boolean mask of the same shape.

In [ ]:
x = np.arange(12).reshape(3,4)
mask = x % 2 == 0
print(x[mask])

**Optimization options**
- - Use boolean masks once; reuse precomputed masks to avoid recomputation; prefer in-place updates with masks if safe.

### Fancy indexing (integer arrays)
Select arbitrary rows/cols by index arrays (always returns a copy).

In [ ]:
y = np.arange(20).reshape(5,4)
rows = np.array([0,2,4])
cols = np.array([1,3,0])
print(y[rows, cols])

**Optimization options**
- - Fancy indexing returns copies; avoid in tight loops; prefer boolean masks/slices for views.

### np.where
Return indices (or choose from values) where condition is True.

In [ ]:
z = np.array([3, -2, 5, -7])
print(np.where(z>0))
print(np.where(z>0, z, 0))

**Optimization options**
- - `where` can both locate and select; combine with broadcasting; prefer `out=` for in-place results.

### np.take / np.put
Take pulls elements along an axis; put writes via flat indices.

In [ ]:
arr = np.arange(10)
print(np.take(arr, [0,3,5]))
arr2 = arr.copy()
np.put(arr2, [1,4], [99,88])
print(arr2)

**Optimization options**
- - `take` is explicit and can be faster along axes; minimize Python list indexing inside loops.

### np.choose
Construct array by choosing from a list of arrays using an index array.

In [ ]:
choices = [np.full(5, -1), np.arange(5), np.arange(5)*10]
idx = np.array([0,1,2,1,0])
print(np.choose(idx, choices))

**Optimization options**
- - `choose` can replace chained `where`; ensure index array is in bounds; pre-allocate choices.

### ravel vs flatten
`ravel()` returns a view when possible; `flatten()` always returns a copy.

In [ ]:
m = np.arange(12).reshape(3,4)
rv = m.ravel(); fl = m.flatten()
print(rv.flags['OWNDATA'], fl.flags['OWNDATA'])

**Optimization options**
- - Prefer `ravel` in pipelines to avoid copies; use `order='F'` or `'C'` for cache-friendly layouts.

### reshape
Change shape without copying when possible; -1 lets NumPy infer dimension.

In [ ]:
n = np.arange(24)
print(n.reshape(2,3,4))
print(n.reshape(3, -1))

**Optimization options**
- - Use `-1` to infer shapes; ensure requested new size matches original total size to avoid errors.

### resize
Resize array in-place (if possible). Can repeat or truncate data.

In [ ]:
p = np.arange(6)
p.resize((3,3))
print(p)

**Optimization options**
- - `resize` can be surprising; prefer `reshape` + explicit padding/truncation for predictability.

### transpose / T
Permute axes; `.T` is shorthand for reversing last two axes on 2D arrays.

In [ ]:
q = np.arange(6).reshape(2,3)
print(q.T)

**Optimization options**
- - Keep track of memory layout after transpose; call `copy(order='C')` if needed for performance-critical ops.

### swapaxes / moveaxis
Rearrange axes: swap two axes or move one axis to a new position.

In [ ]:
r = np.arange(24).reshape(2,3,4)
print(np.swapaxes(r, 0, 2).shape)
print(np.moveaxis(r, 2, 0).shape)

**Optimization options**
- - Use `swapaxes`/`moveaxis` for readability; fewer transposes reduce cache misses.

### squeeze / expand_dims
Remove or add unit dimensions (size 1 axes).

In [ ]:
s = np.zeros((1,3,1,4))
print(np.squeeze(s).shape)
print(np.expand_dims(np.squeeze(s), axis=0).shape)

**Optimization options**
- - Squeezing/remounting dims can unlock broadcasting; avoid unnecessary expansions.

### concatenate / stack / hstack / vstack
Join arrays along existing or new axes.

In [ ]:
a = np.arange(3); b = np.arange(3,6)
print(np.concatenate([a,b]))
print(np.stack([a,b], axis=0))
print(np.hstack([a,b]))
print(np.vstack([a,b]))

**Optimization options**
- - Prefer `concatenate` for existing axes and `stack` for new axes; minimize repeated concatenations—collect then join once.

## Section 3 — Broadcasting and vectorized operations (15 Examples)

### Scalar broadcasting
Scalars broadcast to any array shape for elementwise ops.

In [ ]:
A = np.arange(6).reshape(2,3)
print(A + 10)

**Optimization options**
- - Scalar ops are already optimal; chain ufuncs to reduce temporaries (e.g., `np.add(np.mul(A,2),3)`).

### Row vector broadcast
Shape (1, n) added to (m, n).

In [ ]:
A = np.arange(12).reshape(4,3)
row = np.array([[1,2,3]])
print(A + row)

**Optimization options**
- - Ensure row has shape (1,n) not (n,); use `keepdims` in prior reductions to match shapes.

### Column vector broadcast
Shape (m, 1) added to (m, n).

In [ ]:
A = np.arange(12).reshape(4,3)
col = np.array([[10],[20],[30],[40]])
print(A + col)

**Optimization options**
- - Ensure column shape is (m,1); prefer views over copies when reshaping.

### Higher-dim broadcast
Align trailing dimensions; prepend 1s to smaller shapes.

In [ ]:
X = np.ones((2,1,3))
Y = np.arange(3)
print((X * Y).shape)

**Optimization options**
- - Check shapes from right; use `np.expand_dims` to align when ambiguous.

### np.newaxis/None to align
Insert size-1 axes to enable broadcasting.

In [ ]:
v = np.arange(5)
outer_sum = v[:, None] + v[None, :]
print(outer_sum)

**Optimization options**
- - `None`/`np.newaxis` is zero-cost; prefer over `np.reshape` copies.

### np.broadcast_to
Explicitly broadcast an array to a larger shape (read-only view).

In [ ]:
b = np.array([1,2,3])
B = np.broadcast_to(b, (4,3))
print(B)

**Optimization options**
- - `broadcast_to` avoids materialization; avoid writing to broadcasted views; compute into `out=` buf.

### Vectorized comparisons
Elementwise boolean arrays for filtering and logic.

In [ ]:
x = np.arange(9).reshape(3,3)
mask = (x % 3 == 0) | (x > 6)
print(mask)

**Optimization options**
- - Combine comparisons; use bitwise ops `&`, `|` instead of Python `and`, `or`.

### Masked operations
Use boolean masks to apply ops on subsets without loops.

In [ ]:
x = np.arange(10, dtype=float)
mask = x % 2 == 0
x[mask] *= 1.5
print(x)

**Optimization options**
- - Vectorize masked updates; avoid Python loops; consider `where` to build results without mutation.

### Ufunc with `where`
Compute only where condition holds; else preserve/alternate values.

In [ ]:
a = np.array([-2.0, -0.0, 1.5, 4.0])
res = np.sqrt(a, where=a>=0, out=np.empty_like(a))
print(res)

**Optimization options**
- - `where` plus `out=` minimizes temporary arrays and branches.

### Use `out=` for in-place
Many ufuncs accept `out=` to write results to a preallocated array.

In [ ]:
u = np.arange(5.0)
out = np.empty_like(u)
np.log1p(u, out=out)
print(out)

**Optimization options**
- - Reuse preallocated arrays with `out=`; choose minimal dtype for memory bandwidth gains.

### Keepdims in reductions
Maintain dims for later broadcasting alignment.

In [ ]:
M = np.arange(12).reshape(3,4)
mean_col = M.mean(axis=0, keepdims=True)
print(M - mean_col)

**Optimization options**
- - `keepdims=True` keeps shapes compatible for immediate broadcasting; avoids extra expands later.

### np.einsum for vectorization
Compact, often faster expressions without intermediate arrays.

In [ ]:
A = np.arange(6).reshape(2,3)
B = np.arange(12).reshape(3,4)
C = np.einsum('ik,kj->ij', A, B)
print(C.shape)

**Optimization options**
- - `einsum` can fuse steps; compare with `matmul` and profile on large arrays.

### Outer operations
Use broadcasting to compute pairwise ops across vectors.

In [ ]:
v = np.arange(4)
outer_prod = v[:,None] * v[None,:]
print(outer_prod)

**Optimization options**
- - Broadcasting outer patterns avoids `np.outer` allocation if not needed; or use `np.multiply.outer`.

### np.tensordot
Generalized dot products over specified axes.

In [ ]:
A = np.arange(24).reshape(2,3,4)
B = np.arange(12).reshape(3,4)
T = np.tensordot(A, B, axes=([1,2],[0,1]))
print(T.shape)

**Optimization options**
- - `tensordot` can replace nested matmuls; choose axes carefully; benchmark.

### Avoid Python loops
Vectorize expressions for speed; loops are slow in pure Python.

In [ ]:
x = np.arange(1_0000)
# Vectorized square vs Python loop
vec = x*x
# (Loop version shown for reference only)
# sq = np.empty_like(x)
# for i,val in enumerate(x):
#     sq[i] = val*val
print(vec[:10])

**Optimization options**
- - Prefer vectorized ufunc chains; if still slow, consider `numba` or `Cython` outside NumPy.